In [ ]:
import sys
import os
import argparse
import random

SCRIPT_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Third-party
import numpy as np
import torch
import tensorflow as tf
from datasets import load_dataset

# Communication
from basestation import User, EdgeServer
from user_association import uncertainty_aware_offloading
from utils import (
    generate_rayleigh_coeffs,
    bit_size_text,
)
from config import (
    BANDWIDTH,
    TRANSMIT_POWER,
    NOISE_POWER,
    LOCAL_COMPUTE_CAP,
    EDGE_COMPUTE_CAP,
    MAX_COMPUTE_PER_USER,
    SLM,
    LLM,
    K,
)

# LLM
from model.huggingface_model import HuggingfaceModel
from model import get_model

In [3]:
model = get_model(LLM)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
dataset_ = load_dataset("Muennighoff/babi", split="train")
exclude_tasks = [7, 8, 19]
dataset = dataset_.filter(lambda example: example["task"] not in exclude_tasks)

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
import re


def is_correct(prediction: str, answer: str) -> bool:
    prediction = prediction.strip().lower()
    answer = answer.strip().lower()
    return re.search(rf"\b{re.escape(answer)}\b", prediction) is not None

In [ ]:
passage = dataset[0]["passage"]
question = dataset[0]["question"]
answer = dataset[0]["answer"]
input = (
    f"Instruction: Answer with only one word. No explanation."
    f"Context: {passage.strip()}\n"
    f"Question: {question.strip()}\n"
    f"Answer(only one word):"
)
output, time = model.generate(input)
prediction = output[len(input) :]
tokens = model.tokenizer.encode(input, truncation=True)
wl = len(tokens) * 2 * 8e9
t_comp = wl / LOCAL_COMPUTE_CAP
print(wl)
print(f"expect: {t_comp}")
print(f"real: {time}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


592000000000.0
expect: 0.00650120799472875
real: 0.1215822696685791


: 